In [ ]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.datasets import mnist

import matplotlib.pyplot as plt

kernel_size = 8

def generator_model():
    dropout = 0.4
    dim = 7
    depth = 256
    # In: 100
    # Out: 7 x 7 x 256
    model = Sequential()
    model.add(Dense(dim*dim*depth, input_dim=100))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Reshape((dim, dim, depth)))
    model.add(Dropout(dropout))
    # In: 7 x 7 x 256
    # Out: 14 x 14 x 128
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/2), kernel_size, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 14 x 14 x 128
    # Out: 28 x 28 x 64
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/4), kernel_size, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 28 x 28 x 64
    # Out: 28 x 28 x 1
    model.add(Conv2DTranspose(int(depth/8), kernel_size, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Conv2DTranspose(1, 5, padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    return model

    # Out: 28 x 1


def discriminator_model():
    depth = 64
    dropout = 0.4
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth = 64
    model.add(Conv2D(depth, kernel_size, strides = 2, padding ='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*2, kernel_size, strides = 2, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*4, kernel_size, strides = 2, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*8, kernel_size, strides = 1, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    return model

def adversial_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

def train(epochs = 100, batch_size = 256, save_interval = 20):
    # Initialize data and models
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = input_data.read_data_sets("mnist",one_hot=True).train.images
    X_train = X_train.reshape(-1, 28, 28, 1).astype(np.float32)
    discriminator = discriminator_model()
    generator = generator_model()
    adversial = adversial_model(generator, discriminator) # discriminator is frozen
    d_optim = RMSprop(lr=0.0002, decay=6e-8)
    g_optim = RMSprop(lr=0.0001, decay=3e-8)
#     generator.compile(loss='binary_crossentropy', optimizer="RMSprop")
    adversial.compile(loss='binary_crossentropy', optimizer=g_optim, metrics=['accuracy'])
    discriminator.trainable = True # unfreeze discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim, metrics=['accuracy'])
    
    # Stuff for saving images
    filename = 'mnist.png'
    noise_input = None
    if save_interval > 0:
        noise_input = np.random.uniform(-1, 1, size=[16,100])
    
    # The training loop
    for epoch in range(epochs):
        real_images = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        fake_images = generator.predict(noise)
        combined_images = np.concatenate((real_images, fake_images))
        labels = np.ones([2*batch_size, 1])
        labels[batch_size:, :] = 0
#         labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
#         labels += 0.05 * np.random.random(labels.shape)
        d_loss = discriminator.train_on_batch(combined_images, labels)
                                        
        misleading_targets = np.ones([batch_size, 1])
        random_latent_vectors = np.random.uniform(-1, 1, size=[batch_size, 100])
        discriminator.trainable = False # freeze the discriminator
        a_loss = adversial.train_on_batch(random_latent_vectors, misleading_targets)
        discriminator.trainable = True # unfreeze the discrminator
                                        
        log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
                       
        # Saving sample generated images
        if save_interval>0:
            step = epoch+1
            if step%save_interval==0:
                filename = "mnist_%d.png" % step
                images = generator.predict(noise_input)
                plt.figure(figsize=(10,10))
                for i in range(images.shape[0]):
                    plt.subplot(4, 4, i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image, [28, 28])
                    plt.imshow(image, cmap='gray')
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig(filename)
                plt.close('all')
train()

Using TensorFlow backend.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
co

1: [D loss: 1.317801, acc: 0.500000]  [A loss: 0.720518, acc: 0.082031]
2: [D loss: 0.591307, acc: 0.832031]  [A loss: 0.896997, acc: 0.031250]
3: [D loss: 1.056406, acc: 0.500000]  [A loss: 1.625587, acc: 0.000000]
4: [D loss: 0.397837, acc: 0.705078]  [A loss: 0.483006, acc: 0.972656]
5: [D loss: 0.263441, acc: 0.974609]  [A loss: 0.110946, acc: 1.000000]
6: [D loss: 0.113274, acc: 0.980469]  [A loss: 0.006572, acc: 1.000000]
7: [D loss: 0.185089, acc: 0.941406]  [A loss: 4.638953, acc: 0.000000]
8: [D loss: 0.633718, acc: 0.701172]  [A loss: 0.001468, acc: 1.000000]
9: [D loss: 3.716733, acc: 0.488281]  [A loss: 2.463829, acc: 0.000000]
10: [D loss: 0.358277, acc: 0.796875]  [A loss: 0.484216, acc: 0.921875]
11: [D loss: 0.242495, acc: 0.953125]  [A loss: 0.188000, acc: 1.000000]
12: [D loss: 0.190870, acc: 0.974609]  [A loss: 0.109131, acc: 1.000000]
13: [D loss: 0.167943, acc: 0.978516]  [A loss: 0.114903, acc: 0.992188]
14: [D loss: 0.142169, acc: 0.976562]  [A loss: 0.034440, ac